## 套件

In [2]:
import pandas as pd
from utils import getStockData

### 前處理完的資料

In [3]:
stock_data = getStockData()

In [4]:
stock_data

,code,name,industry,new_industry,date,open,high,low,close,volume,...,PB_TSE,date_year_month,YOY,TAR,foreign_turnover_rate,trust_turnover_rate,broker_turnover_rate,intitution_turnover_rate,short_margin_ratio,margin_short_ratio
0,3088,艾訊,主機板,電腦及週邊,20170103.0,58.3,58.7,58.1,58.4,78.0,...,2.86,201701,-1.75,11.87,0.02,0.0,0.0,0.02,1.60,6250.000000
1,3088,艾訊,主機板,電腦及週邊,20170104.0,59.0,59.0,58.7,59.0,44.0,...,2.89,201701,-1.75,11.87,0.01,0.0,0.0,0.01,1.71,5847.953216
2,3088,艾訊,主機板,電腦及週邊,20170105.0,59.0,59.1,58.6,59.0,62.0,...,2.89,201701,-1.75,11.87,0.01,0.0,0.0,0.01,1.66,6024.096386
3,3088,艾訊,主機板,電腦及週邊,20170106.0,59.0,59.1,58.7,58.8,94.0,...,2.88,201701,-1.75,11.87,0.04,0.0,0.0,0.04,1.66,6024.096386
4,3088,艾訊,主機板,電腦及週邊,20170109.0,58.0,58.3,57.7,58.0,143.0,...,2.84,201701,-1.75,11.87,0.03,0.0,0.0,0.03,1.59,6289.308176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,3088,艾訊,主機板,電腦及週邊,20220322.0,57.7,58.0,57.4,57.8,185.0,...,1.69,202203,10.14,31.96,0.05,0.0,0.0,0.05,0.00,0.000000
1273,3088,艾訊,主機板,電腦及週邊,20220323.0,58.0,59.0,58.0,58.2,278.0,...,1.70,202203,10.14,31.96,0.03,0.0,0.0,0.03,0.00,0.000000
1274,3088,艾訊,主機板,電腦及週邊,20220324.0,58.5,58.5,57.8,58.3,145.0,...,1.71,202203,10.14,31.96,0.05,0.0,0.0,0.05,0.06,166666.666667
1275,3088,艾訊,主機板,電腦及週邊,20220325.0,58.3,59.1,58.1,58.4,273.0,...,1.71,202203,10.14,31.96,0.06,0.0,0.0,0.06,0.06,166666.666667


# 定義要跑模型的 x, y

In [5]:
base_x_variable_columns = [
        'return', 
        'size', 
        'PB_TSE',
        'YOY', 
        'TAR',
]

senti_x_variable_columns = [
        'return', 
        'size', 
        'PB_TSE',
        'YOY', 
        'TAR',
    
        ## 投資人情緒
        'turn_over', 
        'margin_short_ratio', 
        'intitution_turnover_rate',
]



# 應變數 Y

下一期的報酬率

In [6]:
stock_data['return']

0       0.1715
1       1.0274
2       0.0000
3      -0.3390
4      -1.3605
         ...  
1272   -0.5164
1273    0.6920
1274    0.1718
1275    0.1715
1276   -1.7123
Name: return, Length: 1277, dtype: float64

In [7]:
y = stock_data['return'].shift(-1)

# 自變數 x

In [8]:
x = stock_data[base_x_variable_columns]
x

,return,size,PB_TSE,YOY,TAR
0,0.1715,4615.0,2.86,-1.75,11.87
1,1.0274,4663.0,2.89,-1.75,11.87
2,0.0000,4663.0,2.89,-1.75,11.87
3,-0.3390,4647.0,2.88,-1.75,11.87
4,-1.3605,4584.0,2.84,-1.75,11.87
...,...,...,...,...,...
1272,-0.5164,5213.0,1.69,10.14,31.96
1273,0.6920,5249.0,1.70,10.14,31.96
1274,0.1718,5258.0,1.71,10.14,31.96
1275,0.1715,5267.0,1.71,10.14,31.96


# 迴歸模型建立

In [9]:
### https://www.statsmodels.org/dev/install.html
# !pip install statsmodels

In [10]:
import statsmodels.api as sm

### 1. 基本款的控制變數

In [11]:
y_not_na = y[y.notna()].values
x_not_na = x[y.notna()].values
x_not_na = sm.add_constant(x_not_na) ## add_constant 是有考慮常數項

In [12]:
## 建立迴歸模型
base_model = sm.OLS(y_not_na, x_not_na) 
## Fit model
base_res = base_model.fit()

In [14]:
base_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.852
Date:                Wed, 30 Mar 2022   Prob (F-statistic):              0.100
Time:                        16:56:55   Log-Likelihood:                -2032.3
No. Observations:                1276   AIC:                             4077.
Df Residuals:                    1270   BIC:                             4107.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4141      0.518      2.729      0.006       0.398       2.431
x1            -0.0189      0.028     -0.670      0.503      -0.074       0.036
x2            -0.0003      0.000     -2.197      0.028      -0.000   -2.68e-05
x3            -0.1548      0.121     -1.283      0.200      -0.391       0.082
x4             0.0018      0.003      0.684      0.494      -0.003       0.007
x5             0.0042      0.003      1.207      0.228      -0.003       0.011
==============================================================================
Omnibus:                      326.751   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14037.738
Skew:                           0.367   Prob(JB):                         0.00
Kurtosis:                      19.232   Cond. No.                     7.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 2. 考慮投資人情續

In [15]:
x = stock_data[senti_x_variable_columns]
x_not_na = x[y.notna()].values
x_not_na = sm.add_constant(x_not_na) ## add_constant 是有考慮常數項

In [16]:
## 建立迴歸模型
senti_model = sm.OLS(y_not_na,  x_not_na)
## Fit model
senti_res = senti_model.fit()

In [17]:
senti_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.412
Date:                Wed, 30 Mar 2022   Prob (F-statistic):             0.0139
Time:                        16:57:01   Log-Likelihood:                -2027.3
No. Observations:                1276   AIC:                             4073.
Df Residuals:                    1267   BIC:                             4119.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1973      0.571      2.096      0.036       0.077       2.318
x1            -0.0162      0.028     -0.575      0.565      -0.072       0.039
x2            -0.0002      0.000     -1.605      0.109      -0.000    4.42e-05
x3            -0.1399      0.124     -1.132      0.258      -0.382       0.103
x4             0.0016      0.003      0.596      0.551      -0.004       0.007
x5             0.0055      0.004      1.570      0.117      -0.001       0.012
x6            -0.2140      0.085     -2.503      0.012      -0.382      -0.046
x7         -4.564e-06   2.22e-06     -2.056      0.040   -8.92e-06    -2.1e-07
x8             1.3306      0.671      1.984      0.047       0.015       2.646
==============================================================================
Omnibus:                      338.900   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14091.878
Skew:                           0.446   Prob(JB):                         0.00
Kurtosis:                      19.256   Cond. No.                     3.93e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.93e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 預測看看下一期報酬率

In [18]:
stock_data.tail(1)

,code,name,industry,new_industry,date,open,high,low,close,volume,...,PB_TSE,date_year_month,YOY,TAR,foreign_turnover_rate,trust_turnover_rate,broker_turnover_rate,intitution_turnover_rate,short_margin_ratio,margin_short_ratio
1276,3088,艾訊,主機板,電腦及週邊,20220328.0,58.4,58.6,57.3,57.4,298.0,...,1.68,202203,10.14,31.96,0.06,0.0,0.0,0.06,0.07,142857.142857


### 基本款

In [21]:
x = stock_data[base_x_variable_columns]
x_for_pred = sm.add_constant(x.values)[-1]
prediction = base_res.predict(x_for_pred)
prediction

array([0.03904576])

### 考慮投資人情緒

In [22]:
x = stock_data[senti_x_variable_columns]
x_for_pred = sm.add_constant(x.values)[-1]
prediction = senti_res.predict(x_for_pred)
prediction

array([-0.48893757])